# Covid-19 Vaccines in California and their Impacts

In [ ]:
#Hypothesis:
#1.	College towns had an impact on Covid-19 surges in 2020.
#2.	Vaccines have directly impacted California’s case and death rates.


In [3]:
#Dependencies
import pandas as pd
import csv

### Cleaning Data

In [ ]:
# Cleaning Cases Data

In [5]:
# Cleaning Vaccinations Data
vaccine_csv_path = "Data/covid19vaccinesbycounty.csv"
vaccine_df = pd.read_csv(vaccine_csv_path)
vaccine_df.head()

vaccine_csv_path = "Data/covid19vaccinesbycounty.csv"
vaccine_df = pd.read_csv(vaccine_csv_path)

vaccine_df=vaccine_df.drop(columns=['cumulative_total_doses','cumulative_pfizer_doses',
                         'cumulative_moderna_doses','cumulative_jj_doses',
                         'cumulative_at_least_one_dose', 'california_flag'])
vaccine_df=vaccine_df.drop(vaccine_df[(vaccine_df['county']=='All CA Counties')|(vaccine_df['county']=='All CA and Non-CA Counties')|(vaccine_df['county']=='Outside California')|(vaccine_df['county']=='Unknown')].index)
vaccine_df

,county,administered_date,total_doses,pfizer_doses,moderna_doses,jj_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose
511,Alameda,12/15/2020,36,36,0,0,36,36,0,0,36
512,Alameda,12/16/2020,202,202,0,0,202,238,0,0,202
513,Alameda,12/17/2020,396,396,0,0,396,634,0,0,396
514,Alameda,12/18/2020,1766,1765,1,0,1766,2400,0,0,1766
515,Alameda,12/19/2020,1357,1357,0,0,1357,3757,0,0,1357
...,...,...,...,...,...,...,...,...,...,...,...
7761,Yuba,4/19/2021,244,102,142,0,103,5934,141,12839,103
7762,Yuba,4/20/2021,302,79,223,0,177,5987,125,12964,178
7763,Yuba,4/21/2021,694,283,411,0,299,5891,395,13359,299
7764,Yuba,4/22/2021,269,95,174,0,149,5920,120,13479,149


In [ ]:
# Cleaning Hospital Data

In [ ]:
# Cleaning Colleges Data

### Connecting API

### Summary Statistics

## Research Analysis

### 1.	How has the introduction of the Covid vaccine affected case/death rate in California?

### 2.	Do “College” towns play a larger role in case surges?

### 3.	Are there “Vaccination Deserts”? What impact have Covid-19 and the vaccines had in these areas?

### 4.	Can we predict when California will return to “normal”?